In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform
import glob

In [2]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(
        r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")

if platform.system()=='Darwin':
    os.chdir(
        '/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV')

folder=os.getcwd()
data=os.path.join(folder,'Datos')
provisiones=os.path.join(data,'Provisiones')
facturas=os.path.join(data,'Facturas')
tickets=os.path.join(data,'Tickets')
usuarios=os.path.join(data,'Usuarios')
egresos=os.path.join(data,'Egresos')
transacciones=os.path.join(data,'Transacciones')
analisis=os.path.join(folder,'Operacion/Analisis')
resultados=os.path.join(analisis,'Resultados')
bases=os.path.join(analisis,'Bases_analisis')
outputs=os.path.join(analisis,'Outputs')
plots=os.path.join(outputs,'Gráficos')
usuarios_clean=os.path.join(bases,'Usuarios')
transac_clean=os.path.join(bases,'Transacciones')
tickets_clean=os.path.join(bases,'Tickets')
facturas_clean=os.path.join(bases,'Facturas')
egresos_clean=os.path.join(bases,'Egresos')
working=os.path.join(bases,'Bases_trabajo')
codigo=os.path.join(folder,'Operacion/Analisis/Código')
conciliaciones=os.path.join(folder,'Operacion/Operativo/Administrativo/Conciliaciones')

## Importación base de trabajo

In [3]:
list_of_files = glob.glob(os.path.join(outputs,'Base_tickets*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
working_data=pd.read_csv(latest_file, low_memory=False)
working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15058 entries, 0 to 15057
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15058 non-null  int64  
 1   id_ticket              15058 non-null  int64  
 2   departamento           15058 non-null  object 
 3   nombre                 15058 non-null  object 
 4   asunto                 15058 non-null  object 
 5   tecnico                15058 non-null  object 
 6   zona                   15021 non-null  object 
 7   abierto_por            15058 non-null  object 
 8   instal                 15058 non-null  int64  
 9   soporte                15058 non-null  int64  
 10  traslado               15058 non-null  int64  
 11  retiro                 15058 non-null  int64  
 12  mantenimiento          15058 non-null  int64  
 13  facturacion            15058 non-null  int64  
 14  cambio_plan            15058 non-null  int64  
 15  su

In [4]:
#Analisis categoria solicitud tipo de cliente
working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_servicio','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.median)
).reset_index()

,mes_crd,tipo_servicio,categoria,num_tickets,tiempo_respuesta
0,2022-01,corporativo,Instalacion,4,0.326782
1,2022-01,corporativo,Retiro,3,1.858322
2,2022-01,corporativo,Soporte,15,0.137049
3,2022-01,residencial,Instalacion,70,0.589606
4,2022-01,residencial,Retiro,68,0.002708
...,...,...,...,...,...
236,2024-09,corporativo,Soporte,12,1.440666
237,2024-09,residencial,Instalacion,22,2.147581
238,2024-09,residencial,Retiro,43,1.967853
239,2024-09,residencial,Soporte,317,1.034965


In [5]:
#analisis residencia/empresa
summary_res_emp=working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_servicio','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.mean)).reset_index().pivot(
    index='mes_crd',columns=[
        'categoria','tipo_servicio'], 
    values=['num_tickets',
           'tiempo_respuesta']).reset_index()
summary_res_emp.tail()

mes_crd num_tickets                                      \
categoria              Instalacion      Retiro     Soporte Instalacion   
tipo_servicio          corporativo corporativo corporativo residencial   
28             2024-05         3.0         2.0        10.0        69.0   
29             2024-06         4.0         3.0         8.0       194.0   
30             2024-07         6.0         3.0        11.0       164.0   
31             2024-08         6.0         NaN         7.0        20.0   
32             2024-09         3.0         4.0        12.0        22.0   

                                                               \
categoria          Retiro     Soporte    Traslado               
tipo_servicio residencial residencial residencial corporativo   
28                   46.0       148.0        18.0         NaN   
29                   33.0       152.0        23.0         2.0   
30                   47.0       191.0        29.0         NaN   
31                   43.0       214.0        27.0         NaN   
32                   43.0       317.0        18.0         NaN   

              tiempo_respuesta                                      \
categoria          Instalacion      Retiro     Soporte Instalacion   
tipo_servicio      corporativo corporativo corporativo residencial   
28                    2.777940    1.877407    6.860675    4.758159   
29                   17.215807    6.009931    3.999109    4.000273   
30                    7.366375   27.975448    4.046702    5.667399   
31                    2.395432         NaN    0.833236    3.294748   
32                    2.311968    1.851212    2.603126    2.501573   

                                                               
categoria          Retiro     Soporte    Traslado              
tipo_servicio residencial residencial residencial corporativo  
28               6.198658    6.189700    5.100646         NaN  
29               6.160829    5.089437    4.599576    3.759155  
30               4.592572    5.913661    6.080024         NaN  
31               4.419884    2.686543    3.632554         NaN  
32               3.204222    1.961437    3.062317         NaN

In [6]:
#Analisis TV o internet
summary_int_tv=working_data.loc[working_data['cat_tecnico']==1].groupby(
    ['mes_crd','tipo_usuario','categoria']).agg(
    num_tickets=('id_ticket','nunique'), 
    tiempo_respuesta=('tiempo_resol',pd.Series.mean)).reset_index().pivot(
    index='mes_crd',columns=[
        'categoria','tipo_usuario'], 
    values=['num_tickets',
           'tiempo_respuesta']).reset_index()
summary_int_tv.tail(10)

mes_crd num_tickets                                         \
categoria             Instalacion   Retiro  Soporte Traslado Instalacion   
tipo_usuario             internet internet internet internet     paquete   
23            2023-12        43.0     25.0     59.0     13.0        75.0   
24            2024-01        45.0     27.0     74.0     13.0        46.0   
25            2024-02        43.0     26.0     90.0     13.0        44.0   
26            2024-03        27.0     45.0     49.0     20.0        67.0   
27            2024-04        43.0     32.0     50.0     12.0        45.0   
28            2024-05        23.0     28.0     61.0      8.0        49.0   
29            2024-06        47.0     22.0     57.0      9.0       151.0   
30            2024-07        50.0     28.0     61.0     16.0       119.0   
31            2024-08        14.0     16.0     62.0      7.0        12.0   
32            2024-09        16.0     21.0     54.0      8.0         9.0   

                                      tiempo_respuesta                      \
categoria     Retiro Soporte Traslado      Instalacion    Retiro   Soporte   
tipo_usuario paquete paquete  paquete         internet  internet  internet   
23              32.0   141.0     18.0         0.590863  3.136276  1.740514   
24              16.0   121.0     25.0         2.106281  4.511880  4.750357   
25              33.0   215.0     18.0         2.612352  2.270664  7.551347   
26              27.0   114.0      7.0         3.719279  5.667724  2.297816   
27              32.0   133.0     29.0         4.546216  5.700501  3.584558   
28              21.0    97.0     11.0         3.331381  8.737571  5.792350   
29              14.0   104.0     16.0         6.071533  5.881990  5.293988   
30              21.0   140.0     13.0         5.912977  7.885506  5.455845   
31              27.0   161.0     20.0         3.345537  3.137344  3.468371   
32              26.0   275.0     10.0         2.013984  3.545189  2.908578   

                                                                  
categoria     Traslado Instalacion    Retiro   Soporte  Traslado  
tipo_usuario  internet     paquete   paquete   paquete   paquete  
23            3.262285    0.607749  1.722391  1.564064  3.455298  
24            3.080147    2.916657  0.762331  4.000159  3.689055  
25            2.946618    3.124469  4.864577  8.373631  5.586214  
26            3.139028    3.178372  2.408214  2.972677  3.501617  
27            4.811704    8.956164  5.310721  5.271596  5.458179  
28            6.842515    5.341077  2.160092  6.507920  3.760746  
29            4.688988    3.666272  6.566670  4.894939  4.444229  
30            5.819829    5.643436  3.385596  5.971341  6.395976  
31            2.954812    2.824983  5.152764  2.309571  3.869764  
32            3.077948    3.305195  2.743032  1.806828  3.049812

In [7]:
summary_fibra_radio=working_data.loc[
    working_data['cat_tecnico']==1].groupby(
    ['mes_crd', 'categoria', 'tecnologia']).agg(
    tiempo_respuesta=('tiempo_resol',pd.Series.mean), 
    num_tickets=('id_ticket', 'nunique'), 
    encuestas=('encuesta', 'sum')).reset_index().pivot(
    index='mes_crd', columns=[
        'categoria', 'tecnologia'], values=[
        'tiempo_respuesta', 'num_tickets', 'encuestas'
    ]).reset_index()

summary_fibra_radio.tail()

mes_crd tiempo_respuesta                                           \
categoria                Instalacion               Retiro   Soporte             
tecnologia                     fibra      radio     fibra     fibra     radio   
28          2024-05         4.939924   0.001221  6.029198  6.286159  5.825861   
29          2024-06         3.722982  59.130909  6.148255  5.145233  2.110561   
30          2024-07         5.723410        NaN  5.894728  5.819132  5.711474   
31          2024-08         3.213869   0.001701  4.290640  2.426935  7.475683   
32          2024-09         2.478820        NaN  2.966774  1.907049  5.609979   

                                          num_tickets  ...                  \
categoria   Traslado               Retiro Instalacion  ... Traslado Retiro   
tecnologia     fibra      radio     radio       fibra  ...    radio  radio   
28          5.100646        NaN       NaN        68.0  ...      NaN    NaN   
29          4.532342        NaN       NaN       196.0  ...      NaN    NaN   
30          5.518499  22.925775  8.415139       167.0  ...      1.0    2.0   
31          3.632554        NaN  9.977350        24.0  ...      NaN    1.0   
32          3.062317        NaN  6.196939        25.0  ...      NaN    2.0   

             encuestas                                                   
categoria  Instalacion       Retiro Soporte       Traslado       Retiro  
tecnologia       fibra radio  fibra   fibra radio    fibra radio  radio  
28                43.0   0.0    8.0    69.0   1.0     16.0   NaN    NaN  
29               179.0   0.0    5.0    75.0   0.0     23.0   NaN    NaN  
30               131.0   NaN    1.0    24.0   1.0     14.0   0.0    0.0  
31                12.0   0.0    5.0   104.0   0.0     13.0   NaN    0.0  
32                15.0   NaN   13.0    91.0   3.0      9.0   NaN    0.0  

[5 rows x 25 columns]

In [8]:
summary_encuesta_cat=working_data.loc[
    working_data['cat_tecnico']==1
].groupby([
    'mes_crd','categoria'
]).agg(num_tickets=(
    'id_ticket','nunique'), 
       encuesta=(
           'encuesta','sum')).reset_index().pivot(
    columns='categoria',
    index='mes_crd',values=[
        'num_tickets',
        'encuesta']).reset_index()

for i in ['Instalacion','Retiro','Soporte','Traslado']:
    summary_encuesta_cat.loc[
        :,'per_enc_'+i]=summary_encuesta_cat[
        ('encuesta',i)]/summary_encuesta_cat[
        ('num_tickets',i)]
summary_encuesta_cat=summary_encuesta_cat.iloc[:,np.r_[0,-4:0]]

nombre_columnas=[]
for i, x in enumerate(list(summary_encuesta_cat.columns)):
    nombre_columnas.append(x[0])

summary_encuesta_cat.columns=nombre_columnas
summary_encuesta_cat.tail()

,mes_crd,per_enc_Instalacion,per_enc_Retiro,per_enc_Soporte,per_enc_Traslado
28,2024-05,0.597222,0.166667,0.445860,0.888889
29,2024-06,0.905000,0.138889,0.472050,0.958333
30,2024-07,0.764398,0.020000,0.124378,0.466667
31,2024-08,0.402597,0.136364,0.478448,0.481481
32,2024-09,0.711864,0.276596,0.287834,0.590909


In [9]:
for i in [summary_int_tv,summary_fibra_radio,summary_res_emp,]:
    nombres_columnas=[]
    for k, x  in enumerate(list(i.columns)):            
        nombres_columnas.append('_'.join([str(j) for j in list(x)]))
    i.columns=nombres_columnas
    i.rename(columns={'mes_crd__':'mes_crd'}, inplace=True)

In [10]:
working_summary=summary_fibra_radio.merge(
    summary_int_tv, on='mes_crd',how='inner').merge(
    summary_res_emp, on='mes_crd',how='inner').merge(
    summary_encuesta_cat, on='mes_crd',how='inner')
working_summary.tail()

,mes_crd,tiempo_respuesta_Instalacion_fibra,tiempo_respuesta_Instalacion_radio,tiempo_respuesta_Retiro_fibra,tiempo_respuesta_Soporte_fibra,tiempo_respuesta_Soporte_radio,tiempo_respuesta_Traslado_fibra,tiempo_respuesta_Traslado_radio,tiempo_respuesta_Retiro_radio,num_tickets_Instalacion_fibra,...,tiempo_respuesta_Soporte_corporativo,tiempo_respuesta_Instalacion_residencial,tiempo_respuesta_Retiro_residencial,tiempo_respuesta_Soporte_residencial,tiempo_respuesta_Traslado_residencial,tiempo_respuesta_Traslado_corporativo,per_enc_Instalacion,per_enc_Retiro,per_enc_Soporte,per_enc_Traslado
28,2024-05,4.939924,0.001221,6.029198,6.286159,5.825861,5.100646,NaN,NaN,68.0,...,6.860675,4.758159,6.198658,6.189700,5.100646,NaN,0.597222,0.166667,0.445860,0.888889
29,2024-06,3.722982,59.130909,6.148255,5.145233,2.110561,4.532342,NaN,NaN,196.0,...,3.999109,4.000273,6.160829,5.089437,4.599576,3.759155,0.905000,0.138889,0.472050,0.958333
30,2024-07,5.723410,NaN,5.894728,5.819132,5.711474,5.518499,22.925775,8.415139,167.0,...,4.046702,5.667399,4.592572,5.913661,6.080024,NaN,0.764398,0.020000,0.124378,0.466667
31,2024-08,3.213869,0.001701,4.290640,2.426935,7.475683,3.632554,NaN,9.977350,24.0,...,0.833236,3.294748,4.419884,2.686543,3.632554,NaN,0.402597,0.136364,0.478448,0.481481
32,2024-09,2.478820,NaN,2.966774,1.907049,5.609979,3.062317,NaN,6.196939,25.0,...,2.603126,2.501573,3.204222,1.961437,3.062317,NaN,0.711864,0.276596,0.287834,0.590909


In [11]:
filepath=os.path.join(outputs,'Resumen_mes_categorias_tecnicas')
working_summary.to_csv(filepath+'.csv')